In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
## Specify Project Key Metadata

PROJECT_ID = "qwiklabs-gcp-00-d269cced691e"
DATASET_ID = "emergency_ml"
RAW_TABLE = "emergency_calls_raw"
MODEL_NAME = "response_time_model"

GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv"

client = bigquery.Client(project=PROJECT_ID)


In [3]:
## For this practice, it asks for using BigQuery to train model, so will use Bigquery library instead of pandas

dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"

try:
    client.create_dataset(dataset_ref)
    print("Dataset created.")
except Exception:
    print("Dataset already exists.")


Dataset created.


In [4]:
# Data Ingestion, Directly load from GCS bucket location

table_id = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)

load_job = client.load_table_from_uri(
    GCS_URI,
    table_id,
    job_config=job_config
)

load_job.result()
print("Emergency call data loaded.")


Emergency call data loaded.


In [5]:
## Basic Model Data EDA
query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`
LIMIT 10
"""
client.query(query).to_dataframe()


,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37
5,17582,2023-01-01 02:28:50+00:00,Rescue,Downtown,Snowy,Sunday,2,High,25.15,6,28.48
6,21624,2023-01-01 02:44:06+00:00,Rescue,Oakmont,Snowy,Sunday,2,High,3.95,9,19.30
7,36793,2023-01-01 02:53:54+00:00,Fire,Riverside,Sunny,Sunday,2,High,5.87,10,10.72
8,41350,2023-01-01 03:52:33+00:00,Police,Greenfield,Windy,Sunday,3,High,6.66,5,20.55
9,32092,2023-01-01 04:09:23+00:00,Police,Maplewood,Snowy,Sunday,4,High,15.50,13,22.98


In [12]:
## Model Setup, Try Boosted Tree

create_model_query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`
OPTIONS(
  model_type = 'LINEAR_REG',
  input_label_cols = ['response_time']
) AS
SELECT
  *
  EXCEPT (call_id, call_timestamp)
FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`;
"""

In [14]:
client.query(create_model_query).result()
print("BQML model created.")

BQML model created.


In [15]:
## Model Evaluation
evaluate_query = f"""
SELECT *
FROM ML.EVALUATE(
  MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
  (
    SELECT * EXCEPT (call_id, call_timestamp)
    FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`
  )
);
"""

In [16]:
client.query(evaluate_query).to_dataframe()

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.741854,4.73939,0.014721,1.479472,0.831076,0.831078


In [21]:
## Model Inference Using Data in the table

predict_query = f"""
SELECT *
FROM ML.PREDICT(
  MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
  (
    SELECT * EXCEPT (call_id, call_timestamp)
    FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`
  )
);
"""


In [22]:
client.query(predict_query).to_dataframe()


,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,24.405779,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,23.852137,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,19.523975,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,19.614089,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,20.434690,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37
...,...,...,...,...,...,...,...,...,...,...
49995,21.555243,Rescue,Maplewood,Rainy,Saturday,21,Medium,25.37,6,21.76
49996,22.824906,Rescue,Brookfield,Snowy,Saturday,21,Medium,23.22,10,19.12
49997,16.250834,Police,Uptown,Sunny,Saturday,23,Medium,23.20,14,16.88
49998,12.962499,Fire,Greenfield,Rainy,Saturday,23,Medium,5.70,2,14.79
